**Instalando pacotes, importando as bibliotecas, pacotes de pre-processamento NLTK e configurando o Swifter**

In [ ]:
#Importação
#!pip install -U pandas-profiling
#!pip install pyspellchecker
#!pip install spellchecker
#!pip install spellcheck
#!pip install swifter==1.3.4 --user
#!pip install --upgrade --user pandas "dask[complete]"

In [4]:
import tensorflow as tf
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import re
import string
import nltk
import unicodedata
import plotly.express as px
import plotly.graph_objects as go
import time
import swifter
import pickle
from keras.layers import Dropout
from nltk import ngrams
from nltk.corpus import stopwords
from sklearn import preprocessing
from nltk.stem import WordNetLemmatizer
from wordcloud import WordCloud
from nltk.tokenize import word_tokenize, RegexpTokenizer
from itertools import chain

from nltk import bigrams
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import RidgeClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import train_test_split, ParameterGrid
from sklearn import metrics
from sklearn.metrics import accuracy_score
from swifter import set_defaults
from sklearn.metrics import confusion_matrix
from catboost import CatBoostClassifier
from catboost import Pool, cv
from prettytable import PrettyTable
from spellchecker import SpellChecker
import tqdm as notebook_tqd
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Flatten, Dense,MaxPooling1D
from keras.models import Sequential
from keras.utils import to_categorical
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder
from keras.layers import Dropout

In [5]:
#NLTK packages
nltk.download('stopwords') #this package is used for removing stopwords
nltk.download('wordnet') #this package is used for lemmatization
nltk.download('punkt')  #this package is used for tokenization

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\danie\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\danie\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\danie\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [7]:
#Configurando Swifter
set_defaults(
    npartitions = None,
    dask_threshold = 1,
    scheduler = "processes",
    progress_bar = True,
    progress_bar_desc = None,
    allow_dask_on_strings = True,
    force_parallel = True,
)

**Importando a base de Dados para um dataframe**

In [ ]:
#write code to get produtos_pre_tratamento.csv
df = pd.read_csv(r'data/produtos_pre_tratamento')

**Explorando a base de Dados**

A análise da distribuição das categorias dos itens revela uma visão abrangente das diferentes classificações presentes. A categorização dos produtos é essencial para a organização e gestão eficiente do inventário. No contexto atual, foram identificadas quatro principais categorias de produtos:

Embalagem:

Quantidade: 3.416 itens Percentual: 34,72% A categoria "Embalagem" representa o maior segmento, compreendendo cerca de um terço (34,72%) do total de itens. Essa categoria inclui produtos essenciais para acondicionar e proteger os produtos, desempenhando um papel crucial na logística e no transporte.

EPI (Equipamentos de Proteção Individual):

Quantidade: 2.673 itens Percentual: 27,16% A categoria "EPI" é significativa, abrangendo aproximadamente 27,16% do total de itens. Esses equipamentos são essenciais para garantir a segurança e o bem-estar dos trabalhadores em diversas atividades, promovendo um ambiente laboral mais seguro.

Papelaria:

Quantidade: 2.364 itens Percentual: 24,02% A categoria "Papelaria" representa cerca de 24,02% do total de itens. Itens de papelaria são essenciais para as operações administrativas e organizacionais, sendo elementos básicos em escritórios e ambientes de trabalho.

Hidráulico:

Quantidade: 1.387 itens Percentual: 14,10% A categoria "Hidráulico" abrange aproximadamente 14,10% do total de itens. Itens hidráulicos são cruciais para sistemas e processos relacionados à distribuição e controle de água e fluidos.

Essa análise permite uma compreensão clara da composição do inventário, destacando a importância de cada categoria na estrutura geral do estoque. Isso pode ser valioso para a tomada de decisões estratégicas, otimização do estoque e atendimento às necessidades específicas do negócio.

In [9]:
#Listagem das categorias e as quantidades com percentual
contagem_categoria = df['Categoria'].value_counts()
total = len(df)  #Total de itens

for categoria, quantidade in contagem_categoria.items():
    porcentagem = (quantidade / total) * 100
    print(f"Categoria: {categoria}, Quantidade: {quantidade}, Percentual: {porcentagem:.2f}%")

Categoria: Embalagem, Quantidade: 3416, Percentual: 34.72%
Categoria: Epi, Quantidade: 2673, Percentual: 27.16%
Categoria: Papelaria, Quantidade: 2364, Percentual: 24.02%
Categoria: Hidráulico, Quantidade: 1387, Percentual: 14.10%


**Início do Pré Processamento**

Criação de funções de pré processamento com as libs SpellChecker, NLTK e unicodedata

In [ ]:
dfManipulated = df.copy() #Copia do dataset original

In [ ]:
# Calcular o total de itens
total = len(df)

# Calcular o percentual e a quantidade para cada categoria
porcentagens = []
quantidades = []
categorias = []
labels = []  # Lista para armazenar os rótulos
for categoria, grupo in df.groupby('Categoria'):
    quantidade = len(grupo)
    porcentagem = (quantidade / total) * 100
    porcentagens.append(porcentagem)
    quantidades.append(quantidade)
    categorias.append(categoria)
    labels.append(f'{categoria}<br>({quantidade} itens)')

# Cores para cada categoria
cores = ['blue', 'orange', 'green', 'red']

# Ajustar os raios para 80% da área
raio_externo = 0.4  # Raio externo para 80% da área
raio_interno = raio_externo - 0.02  # Raio interno para criar o "anel"

# Criar o gráfico de pizza
fig = go.Figure(data=[go.Pie(labels=labels,
                             values=porcentagens,
                             textinfo='label+percent',  # Exibir rótulos e percentuais
                             hole=raio_interno,  # Raio interno
                             marker=dict(colors=cores))])

# Adicionar título e ajustar layout
fig.update_layout(title_text=f'Distribuição dos Itens por Categorias - Total: {total}',
                  margin=dict(t=50, b=50, l=50, r=50))  # Margens para remover espaços desnecessários

# Mostrar o gráfico
fig.show()

In [ ]:
spell = SpellChecker(language='pt') #Instanciando o objeto SpellChecker

#A função abaixo utilizando a biblioteca SpellChecker corrige as palavras que estão escritas erradas.
def pyspellchecker(text):
    word_list = word_tokenize(text, language = 'portuguese')
    word_list_corrected = []
    for word in word_list:
        if word in spell.unknown(word_list) and len(word) > 3:
            word_corrected = spell.correction(word)
            if word_corrected == None:
                word_list_corrected.append(word)
            else:
                word_list_corrected.append(word_corrected)
        else:
            word_list_corrected.append(word)
    text_corrected = " ".join(word_list_corrected)
    return text_corrected 

In [ ]:
#A função abaixo utilizando a biblioteca unicodedata, remove os acentos das palavras.
def strip_accents(s):
   return ''.join(c for c in unicodedata.normalize('NFD', s)
                  if unicodedata.category(c) != 'Mn')

In [ ]:
#A função abaixo utiliza a biblioteca NLTK para remover as stopwords, pontuações, caracteres especiais, palavras com menos de 2 caracteres e aplica a lemmatization.
def limpar_texto(text):
   text = str(text)

    # Remover caracteres non-ascii
   text = ''.join(caracter for caracter in text if ord(caracter) < 128)

    # Convertendo para lower case
   text = text.lower()

    # Removendo pontuação por expressão regular
   regex = re.compile('[' + re.escape(string.punctuation) + '\\r\\t\\n]')
   text = regex.sub(' ', str(text))

   text = re.sub('[^A-Za-z ]+', '', text)

    # Carregando stopwords em português
   portuguese_stops = set(stopwords.words('portuguese'))

    # Removendo stopwords em português
    # Mantendo somente palavras que não são consideradas stopwords
   text = ' '.join(palavra for palavra in text.split() if palavra not in portuguese_stops)

    # Remover palavras com menos de 2 caracteres
   words = text.split()
   text = [word for word in words if len(word) > 2 or word == ' ']
   text = ' '.join(text)

    # Corrigindo erros gramaticais
   text = pyspellchecker(text)

    # Criando a estrutura baseada em uma wordnet para lemmatization
   wordnet_lemmatizer = WordNetLemmatizer()
   
   # Aplicando Lemmatization
   text = ' '.join(wordnet_lemmatizer.lemmatize(palavra) for palavra in text.split())

    # Remover palavras com menos de 2 caracteres residuais
   words = text.split()
   text = [word for word in words if len(word) > 2 or word == ' ']
   text = ' '.join(text)

   text = strip_accents(text)

   return text

In [ ]:
%%time
print('Tratanto texto...')
dfManipulated['descricao'] = dfManipulated['descricao'].map(limpar_texto)
print('Texto tratado')

**Exporta o dataframe pre processado para formato .csv e em seguida atribui-se a variavel df esse novo .csv pre processado**

In [ ]:
# Exporta o DataFrame pre processado para um arquivo .csv
dfManipulated.to_csv('data/dfPreProcessado.csv', index=False)
df = pd.read_csv(r'data/dfPreProcessado.csv')

In [ ]:
#a função a seguir é responsável por exibir a descrição de um produto aleatório ou filtrado, caso deseje passar no parametro.
def exibir_descricao(filtro=None):
    if filtro:
        descricao_filtrada = df[df['descricao'].str.contains(filtro, case=False)]
        descricao_list = descricao_filtrada['descricao'].tolist()
    else:
        descricao_list = df['descricao'].sample(10).tolist()  # Seleciona 10 itens aleatórios

    # Criar uma tabela com alinhamento à esquerda
    tabela = PrettyTable()
    tabela.field_names = ['Descrição']
    tabela.align['Descrição'] = 'l'  # Alinha à esquerda

    # Adicionar as descrições à tabela
    for descricao in descricao_list:
        tabela.add_row([descricao])

    # Exibir a tabela
    print(tabela)

**Identificação e criação de csv's para Frases repetidas, frases que aparecem em mais de uma categoria e palavras iguais.**

In [ ]:
#CRIAÇÃO DE CSV COM FRASES REPETIDAS

# Limpeza dos espaços no início e final das frases
df['descricao'] = df['descricao'].str.strip()

# Identificação de frases repetidas
frases_repetidas = df[df.duplicated('descricao', keep=False)]

# Contagem das frases repetidas
contagem_frases_repetidas = frases_repetidas['descricao'].value_counts()

# Criar um DataFrame com a quantidade e descrição repetida
df_contagem = pd.DataFrame({'descricao_repetida': contagem_frases_repetidas.index,
                            'quantidade': contagem_frases_repetidas.values})

# Adicionar coluna com frase e quantidade
df_contagem['frase_quantidade'] = df_contagem.apply(lambda row: f"{row['descricao_repetida']}, Quantidade: {row['quantidade']}", axis=1)

# Exportar para CSV
df_contagem.to_csv('data/frases_repetidas.csv', columns=['frase_quantidade'], index=False)
print("Csv Frases repetidas e suas contagens criado na pasta data.")

In [ ]:
#VERIFICANDO FRASES QUE APARECEM EM MAIS DE UMA CATEGORIA
# Limpeza dos espaços no início e final das frases
df['descricao'] = df['descricao'].str.strip()

# Contar quantas vezes cada frase aparece para diferentes categorias
contagem_frases_por_categoria = df.groupby(['descricao', 'Categoria']).size().reset_index(name='contagem')

# Filtrar frases que aparecem em mais de uma categoria
frases_em_mais_de_uma_categoria = contagem_frases_por_categoria[contagem_frases_por_categoria.duplicated('descricao', keep=False)]

# Exibir as frases que aparecem em mais de uma categoria
print("Frases que aparecem em mais de uma categoria:")
print(frases_em_mais_de_uma_categoria)
print("Criado o CSV frases que aparecem em mais de uma categoria")
frases_em_mais_de_uma_categoria.to_csv('data/frases_em_mais_de_uma_categoria.csv', index=False)

In [ ]:
#essa função retorna a quantidade de ocorrências de 'GALV' (maiúscula ou minúscula) com a possibilidade de um ponto após a última letra
def contar_ocorrencias_galv(texto):
    padrao = re.compile(r'(?i)GALV\.?')
    return len(padrao.findall(texto))

# Aplicar a função a cada linha da coluna 'descricao'
df['contagem_galv'] = df['descricao'].apply(contar_ocorrencias_galv)

# Calcular o total de ocorrências de 'GALV' (maiúscula ou minúscula) com a possibilidade de um ponto após a última letra
total_ocorrencias_galv = sum(df['contagem_galv'])

# Exibir o total de ocorrências
print('Total de ocorrências de "GALV" (com ou sem ponto após a última letra):', total_ocorrencias_galv)

In [ ]:
# Função para identificar palavras iguais em sequência e contar sua ocorrência
def identificar_palavras_iguais(texto):
    padrao = r'\b(\w+)\s+\1\b'
    palavras_iguais = re.findall(padrao, texto, flags=re.IGNORECASE)
    quantidade = len(palavras_iguais)
    
    return palavras_iguais, quantidade

# Identificar palavras iguais em sequência e contar a quantidade para cada descrição
df[['palavras_iguais', 'quantidade']] = df['descricao'].apply(lambda x: pd.Series(identificar_palavras_iguais(x), index=['palavras_iguais', 'quantidade']))

# Filtrar e exibir apenas as linhas com palavras duplicadas
df_com_palavras_duplicadas = df[df['quantidade'] > 0]

# Criar um DataFrame para o CSV contendo as palavras duplicadas e a frase correspondente
df_csv = pd.DataFrame({'Palavra_Duplicada': df_com_palavras_duplicadas['palavras_iguais'].apply(lambda x: x[0]),
                       'Frase': df_com_palavras_duplicadas['descricao']})

# Salvar o DataFrame em um arquivo CSV
df_csv.to_csv('data/palavras_duplicadas.csv', index=False)

print("Arquivo CSV 'palavras_duplicadas.csv' criado com sucesso.")

**Representação gráfica das palavras duplicadas**

In [ ]:
# Carregar o arquivo CSV em um DataFrame
df = pd.read_csv('data/palavras_duplicadas.csv')

# Calcular a quantidade de ocorrências para cada palavra duplicada
contagem_palavras = df['Palavra_Duplicada'].value_counts()

# Filtrar para palavras com quantidade maior que 50 e menor ou igual a 50
maior_que_50 = contagem_palavras[contagem_palavras > 50]
menor_ou_igual_a_50 = contagem_palavras[contagem_palavras <= 50]

# Criar o gráfico para quantidade maior que 50
fig_maior_que_50 = go.Figure(go.Bar(
    y=maior_que_50.index,
    x=maior_que_50.values,
    orientation='h',
    text=maior_que_50.values,
    textposition='auto',
    hoverinfo='text',
    marker=dict(color='royalblue')
))

fig_maior_que_50.update_layout(
    title='Quantidade de Palavras Duplicadas (Quantidade > 50)',
    xaxis_title='Quantidade',
    yaxis_title='Palavra Duplicada',
    template='plotly_white'
)

# Criar o gráfico para quantidade menor ou igual a 50
fig_menor_ou_igual_a_50 = go.Figure(go.Bar(
    y=menor_ou_igual_a_50.index,
    x=menor_ou_igual_a_50.values,
    orientation='h',
    text=menor_ou_igual_a_50.values,
    textposition='auto',
    hoverinfo='text',
    marker=dict(color='purple')
))

fig_menor_ou_igual_a_50.update_layout(
    title='Quantidade de Palavras Duplicadas (Quantidade <= 50)',
    xaxis_title='Quantidade',
    yaxis_title='Palavra Duplicada',
    template='plotly_white'
)

# Exibir os gráficos
fig_maior_que_50.show()
fig_menor_ou_igual_a_50.show()

**Remoção de palavras iguais em sequencia. Ex: Cano Hidraulico Hidraulico**

In [ ]:
#Remover plavras iguias em sequencia exemplo cor cor
def remover_palavras_iguais_em_sequencia(texto):
    padrao = r'\b(\w+)\b\s+\1\b'
    texto_sem_repeticao = re.sub(padrao, r'\1', texto, flags=re.IGNORECASE)
    return texto_sem_repeticao

df = pd.read_csv(r'data/dfPreProcessado.csv')

# Aplicar a função para remover palavras iguais em sequência na coluna "descricao"
df['descricao'] = df['descricao'].apply(remover_palavras_iguais_em_sequencia)
print("Palavras iguais em sequência removidas")

In [ ]:
# Função para identificar palavras-chave em uma descrição manual informe a palavra no array palavras_chave 
def identificar_palavras_chave(descricao):
    palavras_chave = ['filial', 'vender', 'fort', 'miguel','uso']
    presentes = {palavra: descricao.lower().count(palavra) for palavra in palavras_chave}
    return presentes

# Calcular a quantidade de cada palavra-chave encontrada
df['palavras_chave_quantidade'] = df['descricao'].apply(identificar_palavras_chave)

# Calcular a soma da quantidade de cada palavra-chave
soma_palavras_chave = df['palavras_chave_quantidade'].apply(pd.Series).sum().sort_values()

# Criar um gráfico de barras verticais com plotly
fig = go.Figure(go.Bar(
    y=soma_palavras_chave.index,
    x=soma_palavras_chave.values,
    text=soma_palavras_chave.values,
    textposition='auto',
    marker_color='royalblue',
    orientation='h',
))

fig.update_layout(
    title='Quantidade de cada Palavra-chave',
    xaxis_title='Palavra-chave',
    yaxis_title='Quantidade',
    template='plotly_white'
)

# Exibir o gráfico
fig.show()

In [ ]:
df = pd.read_csv(r'data/dfPreProcessado.csv')

# Função para contar palavras de até 3 caracteres
def contar_palavras_ate_3_caracteres(texto):
    palavras = texto.lower().split()
    palavras_ate_3 = [palavra for palavra in palavras if len(palavra) == 3]
    return Counter(palavras_ate_3)


# Calcular a contagem de palavras de até 3 caracteres em todas as descrições
contagem_palavras = Counter()
for descricao in df['descricao']:
    contagem_palavras += contar_palavras_ate_3_caracteres(descricao)

# Filtrar palavras com quantidade maior que 500 ou maior que 100
palavras_maior_500 = {palavra: contagem_palavras[palavra] for palavra in contagem_palavras if contagem_palavras[palavra] > 500}
palavras_maior_100 = {palavra: contagem_palavras[palavra] for palavra in contagem_palavras if 100 < contagem_palavras[palavra] <= 500}
palavras_menor_100 = {palavra: contagem_palavras[palavra] for palavra in contagem_palavras if contagem_palavras[palavra] <= 100}

# Criar um gráfico de barras para quantidade maior que 500
fig_maior_500 = go.Figure(go.Bar(
    y=list(palavras_maior_500.keys()),
    x=list(palavras_maior_500.values()),
    text=list(palavras_maior_500.values()),
    textposition='auto',
    marker_color='royalblue',
    orientation='h',
))

fig_maior_500.update_layout(
    title='Quantidade de Palavras com mais de 500 ocorrências',
    xaxis_title='Palavra',
    yaxis_title='Quantidade',
    template='plotly_white'
)

# Criar um gráfico de barras para quantidade entre 100 e 500
fig_entre_100_e_500 = go.Figure(go.Bar(
    y=list(palavras_maior_100.keys()),
    x=list(palavras_maior_100.values()),
    text=list(palavras_maior_100.values()),
    textposition='auto',
    marker_color='purple',
    orientation='h',
))

fig_entre_100_e_500.update_layout(
    title='Quantidade de Palavras com mais de 100 e até 500 ocorrências',
    xaxis_title='Palavra',
    yaxis_title='Quantidade',
    template='plotly_white'
)

# Criar um gráfico de barras para quantidade menor que 100
fig_menor_100 = go.Figure(go.Bar(
    y=list(palavras_menor_100.keys()),
    x=list(palavras_menor_100.values()),
    text=list(palavras_menor_100.values()),
    textposition='auto',
    marker_color='green',
    orientation='h',
))

fig_menor_100.update_layout(
    title='Quantidade de Palavras com até 100 ocorrências',
    xaxis_title='Palavra',
    yaxis_title='Quantidade',
    template='plotly_white'
)

# Exibir os gráficos
fig_maior_500.show()
fig_entre_100_e_500.show()
fig_menor_100.show()

**Remoção de palavras indesejadas e atualização do dataframe**

In [ ]:
stop_words_personalizadas = ['ete', 'ftc', 'xxm','ref'
                             'saw', 'shp','uso',
                             'filial','preto','preta','vermelho','vermelha', 'azul', 'verde', 'amarelo', 'laranja']
def remover_stop_words(texto):
    # Padroniza a lista de stop words para garantir que a comparação seja insensível a maiúsculas e minúsculas
    stop_words = [re.escape(word.lower()) for word in stop_words_personalizadas]

    # Cria uma expressão regular para encontrar as stop words
    padrao = r'\b(?:' + '|'.join(stop_words) + r')\b'
    
    # Remove as stop words do texto
    texto_sem_stop_words = re.sub(padrao, '', texto, flags=re.IGNORECASE)
    return texto_sem_stop_words


df = pd.read_csv(r'data/dfPreProcessado.csv')
df['descricao'] = df['descricao'].apply(remover_stop_words)

**Substituição em massa dos das palavras**

In [ ]:
def replace_em_massa_dataframe(df, coluna, vetor_valores_substituir, vetor_novos_valores):
    
    # Garante que os tamanhos dos vetores são iguais
    if len(vetor_valores_substituir) != len(vetor_novos_valores):
        raise ValueError("Os vetores de substituição devem ter o mesmo tamanho")

    # Cria um dicionário de substituições
    substituicoes = dict(zip(vetor_valores_substituir, vetor_novos_valores))

    # Aplica as substituições na coluna "descricao"
    df[coluna] = df[coluna].replace(substituicoes, regex=True)

    return df

# Passe as palavras neste array 
vetor_valores_substituir = ["canetass",
                            "luvas",
                            "filial",
                            "GALV",
                          
                            ]
vetor_novos_valores = ["caneta", 
                       "luva",
                       "",
                       "galvanizada",
                       
                     ]

df = replace_em_massa_dataframe(df, 'descricao', vetor_valores_substituir, vetor_novos_valores)

**Gerando o arquivo final que será utilizado nos modelos de classificação**

In [ ]:
df_copia = df
df_copia['descricao'] = df_copia['descricao'].str.strip()

# Remover frases duplicadas
df_sem_duplicatas = df_copia.drop_duplicates(subset=['descricao'])

#Exporta o csv resultante
df_sem_duplicatas.to_csv('data/df_sem_duplicatas.csv', index=False)

# Exibir DataFrame sem frases duplicadas
print("DataFrame sem frases duplicadas:")
print(df_sem_duplicatas)